# 01-3 AI·LLM·에이전트 활용 체험 실습
## Rule-based Planner + LLM Reasoner 혼합 구조

### 학습목표

- 동일한 문제를 전통적 AI / LLM / AI 에이전트 관점에서 비교할 수 있다.  
- LLM의 “응답 생성”과 에이전트의 “행동 수행” 차이를 체감한다.  
- Rule-based Planner + LLM Reasoner 혼합 구조의 필요성을 이해한다.  
- 이후 LangChain·Tool Calling·멀티 에이전트 학습의 기초 개념을 확보한다.  

### 들어가며

- **질문: “회의록 요약과 후속 업무를 누가, 어떻게 처리할까?”**  

현실의 업무 흐름은 다음과 같다.  
- 회의 내용을 정리한다  
- 핵심 결론과 할 일을 도출한다  
- 필요한 계산을 수행한다  
- 결과를 문서로 저장하거나 공유한다  

> 이 과정은 단순 답변 생성이 아니라  
> 여러 단계를 계획하고 실행하는 ‘에이전트형 문제'에 해당한다.  


### 01-3-1 전통적 AI 관점 (Rule-based 처리)

#### 개념 시뮬레이션

- 방식: 미리 정의된 규칙에 따라 처리  
- 예:  
    - 키워드 포함 문장 추출  
    - 고정된 To-do 템플릿 출력  

```text  
IF "예산" 포함 → 예산 항목 기록
IF "일정" 포함 → 일정 항목 기록
```

#### 특징과 한계

- ✔ 예측 가능, 안정적  
- ❌ 문맥 이해 불가  
- ❌ 새로운 요청에 취약  
- ❌ 작업 확장 불가  

> 토의 질문
> “이 방식으로 회의 요약 + 계산 + 파일 저장을 유연하게 처리할 수 있을까?”


### 01-3-2 LLM 관점 (Reasoner only)

#### 실습: LLM에게 회의록 처리 요청

- 프롬프트 예시  

```text  
다음 회의 내용을 요약하고,
핵심 To-do 3개를 정리해줘.

[회의 내용]
- 마케팅 캠페인 일정 논의
- 예산은 3천만 원
- SNS 홍보 강화 필요
```

#### 결과 특징

- ✔ 자연스러운 요약  
- ✔ 문맥 이해  
- ❌ 계산 실행 불가  
- ❌ 파일 저장 불가  
- ❌ 후속 작업 자동화 불가  

> 핵심 정리  
> LLM은 잘 생각하지만, 직접 행동하지는 못한다  


### 01-3-3 AI 에이전트 관점(Rule-based Planner + LLM Reasoner 혼합 구조)

#### (1) 에이전트 구조 개요

```text  
[사용자 요청]
      ↓
Rule-based Planner
      ↓
LLM Reasoner (요약·추론)
      ↓
Tool Executor
  - 계산
  - 파일 저장
```

- Planner: 전체 작업 흐름을 통제  
- LLM: 요약·판단·언어 추론 담당  
- Tool: 실제 행동 수행  

In [ ]:
# 라이브러리 불러오기
import json, re
from transformers import pipeline

# 사용할 LLM 모델 지정
MODEL_NAME = "google/flan-t5-base"
llm = pipeline("text2text-generation", model=MODEL_NAME, max_new_tokens=256)

# LLM 호출함수 정의
def llm_generate(prompt: str) -> str:
    out = llm(prompt, do_sample=False)
    return out[0]["generated_text"].strip()

# 정상 로드 확인
print("LLM ready:", MODEL_NAME)

In [ ]:
# 실행 모듈: 할일 요약 함수
def make_summary_todos(meeting_text: str) -> str:
    prompt = f"""
다음 회의록을 한국어로 4줄 이내 요약하고,
To-do를 3개 bullet로 작성하라.

회의록:
{meeting_text}
"""
    result = llm_generate(prompt)
    if not result or len(result.strip()) < 20:
        # 예외 처리 (안전장치)
        return "(요약 생성 실패)\n\n- (To-do) 후속 조치 1\n- (To-do) 후속 조치 2\n- (To-do) 후속 조치 3"
    return result

In [ ]:
# 실행 모듈: 계산기
def calculator(expression: str) -> float:
    return eval(expression)

# 실행 모듈: 파일 저장
def save_text(filepath: str, content: str):
    import os
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

#### (2) Planner 역할 (규칙 기반)  

```json  
{
  "steps": [
    {"tool": "make_summary_todos"},
    {"tool": "calculator"},
    {"tool": "save_text"}
  ]
}
```

- ✔ Planner는 LLM 출력이 불안정해도 흐름을 유지  
- ✔ 실패 시 폴백 전략 가능  

In [ ]:
def planner():
    return {
        "steps": [
            {"tool": "make_summary_todos"},
            {"tool": "calculator", "expression": "30000000 * 0.15"},
            {"tool": "save_text", "filepath": "output/agent_result.txt"}
        ],
        "final_message": "회의록 처리를 완료했습니다."
    }

#### (3) LLM Reasoner 역할

- 회의록 요약  
- To-do 생성  
- 자연어 처리 전담  

```python
def make_summary_todos(meeting_text):
    # LLM은 '내용 생성'만 담당
```

> 중요  
> LLM은 계획자도, 실행자도 아님  
> → “생각하는 두뇌” 역할  

In [ ]:
# AI 에이전트 실행 함수
def run_agent(meeting_text: str):
    plan = planner()
    summary = make_summary_todos(meeting_text)
    calc_result = calculator("30000000 * 0.15")
    final_content = f"{summary}\n\n[계산] 3천만 원의 15% = {calc_result} 원"
    save_text("output/agent_result.txt", final_content)
    print("FINAL:", plan["final_message"])
    print("\n--- 저장된 파일 ---")
    print(final_content)

In [ ]:
# Main 셀
MEETING_TEXT = """
- 마케팅 캠페인 일정 논의
- 예산은 3천만 원
- SNS 홍보 강화 필요
"""

run_agent(MEETING_TEXT)

#### (4) 실행 결과 예시 (개선된 출력 목표)

```text  
[회의 요약]
- 2분기 마케팅 캠페인 추진
- 예산 3천만 원 배정
- SNS 홍보 강화 결정

[To-do]
- SNS 콘텐츠 기획안 작성
- 예산 집행 일정 수립
- 외주 업체 검토

[계산]
3천만 원의 15% = 4,500,000원
```

> 이것이 AI 에이전트가 “일을 처리한다”는 의미  

## 학습정리
Rule-based Planner + LLM Reasoner 구조가 가장 안정적이다.